![](slides/tiny_ml_cover.svg)

![](slides/ml_morphology.svg)

![](slides/mlops_graph.svg)

![](slides/edge_devices.svg)


 <tr>
    <td> <img src="./slides/spark_micro_mod_pico_1.jpg" alt="Drawing" style="height: 50%;width:100%;" align="right"/> </td>
    <td> <img src="./slides/micro_mod_pico_2.jpg" alt="Drawing" style="height: 50%; width:100%" align="left"/> </td>
    </tr>


In [1]:
# !git clone https://github.com/wandb/tiny-ml.git

In [72]:
import tensorflow as tf
import wandb
from pathlib import Path
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import numpy as np
import sounddevice as sd
import wave
from pvrecorder import PvRecorder
import time 
from IPython.display import HTML
from IPython.display import IFrame

In [3]:
# some utilities that weve made in 
import recorder
from utils.data_processing import augmenter,log_wandb_artifact,plot_spectrogram,segment,read_wav,Arm_spect
from utils.create_sweep import create_sweep

In [4]:
# %load utils/data_processing.py

In [173]:
sample  = recorder.record.Response(512)
# set the time we want to sample sound
sample.set_time(delta_sec=1)
# defie wandb target projcts
sample.set_run(project='wake_word_detection',entity='tiny-ml',group='yes_no',name='sound_recording')
# set classes
sample.get_classes(('yes','background'))

In [73]:
tqdm_args = dict(leave=True,position=0)
tqdm_colours = (f'#{r:02x}{g:02x}{b:02x}'for r,g,b in [(int(255*s), 3, int(255*s)) for s in np.linspace(1,0,100)])
run = wandb.init(entity='tiny-ml',project = 'wake_word_detection', group='Data')

In [74]:
recorder = PvRecorder(device_index=0, frame_length=512)
sound = []
recorder.start()
t_0 = time.time()
while time.time()-t_0<4:
    frame = recorder.read()
    sound.extend(frame)
recorder.stop()

In [75]:
data = np.array(sound).astype(np.int16)
t_step = np.arange(0, len(data))
fig = px.line(x=t_step, y=data, title='a sound wave')
run.log({'sound_wave': fig})

### Showing a Run in Notebook

In [76]:
run

In [77]:
path = './data/yes/yes_record 102.wav'
run.log({'test_sound':wandb.Audio(path)})
run

# The basis of our signal processing is using Fast Fourier Transform (FFT)
> - [here](https://www.youtube.com/watch?v=spUNpyF58BY) is an amazing video on how FFT works. 
> - quickly go through getting from FFT to a Spectrogram. 
>  ![](./slides/200px-Fourier_in_his_coat_of_prefect.jpg)

In [78]:
fft = np.fft.fft(data).astype(np.int16)
t_steps = np.arange(0,len(fft))
fft = np.stack([fft,t_step]).T
df = pd.DataFrame(fft, columns = ['Faurrier_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Faurrier_Value', title='Fourrier Transform')
run.log({'Fouriere Transform':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_41368/660930182.py:1: ComplexWarning:

Casting complex values to real discards the imaginary part



In [79]:
run

## FFT can also be inverted, we can get sound back

In [81]:
fft = np.fft.fft(data)
roll = np.roll(fft,200)
ifft = np.fft.ifft(roll).astype(np.int16)
t_step = np.arange(len(ifft))
ifft = np.stack([ifft,t_step]).T
df = pd.DataFrame(ifft, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', 
              y='Frequecy_Value', 
              title='Inverse Fourrier Transform to Get sound Back')
run.log({'Inverset Fouriere':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_41368/1994990191.py:3: ComplexWarning:

Casting complex values to real discards the imaginary part



In [82]:
run

## Down Sample using FFT as an example of Signal Processing

In [83]:
# and examper of downsampling using FFT
roll = np.roll(fft,15)
ifft_ten = np.fft.ifft(roll)
ifft_ten = ((2**(16-4)) * ifft_ten/ifft_ten.max()).astype(np.int16)
ifft_ten = np.stack([ifft_ten,t_step]).T
df = pd.DataFrame(ifft_ten, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Frequecy_Value', title='Dowsampling using Fast Fourrier Transform (FFT)')
run.log({'Down_Sampled_Sound':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_41368/3482508054.py:4: ComplexWarning:

Casting complex values to real discards the imaginary part



In [84]:
run

In [85]:
run.finish()
run = wandb.init(id=run.id,
                 entity='tiny-ml',
                 project = 'wake_word_detection ',
                 group='Data',
                 resume='must')

### ARM spectrogram using FFT 🦾


In [86]:
category = 'yes'
idx = 10
sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
sound = sound.astype(np.float32, order='F') / 32768.0
get_arm_spectrogram = Arm_spect().get_arm_spectrogram
get_arm_spectrogram(sound)

array([[0.5     , 0.3125  , 0.125   , ..., 0.      , 0.      , 0.      ],
       [0.671875, 0.375   , 0.0625  , ..., 0.015625, 0.      , 0.      ],
       [0.78125 , 0.390625, 0.03125 , ..., 0.      , 0.      , 0.      ],
       ...,
       [1.109375, 0.53125 , 0.125   , ..., 0.015625, 0.      , 0.      ],
       [0.96875 , 0.375   , 0.03125 , ..., 0.015625, 0.      , 0.      ],
       [0.921875, 0.34375 , 0.0625  , ..., 0.      , 0.      , 0.      ]],
      dtype=float32)

In [87]:
IFrame('https://wandb.ai/tiny-ml/wake_word_detection/reports/Walking-Rubbix-Cube--VmlldzozNDU0NTI5', width=1900,
       height=800)

### Showing output of custom function optimized for arm processor

In [88]:
category = 'yes'
sound = read_wav(f'./data/{category}/yes_record 119.wav')
print(sound)
sound = sound.astype(np.float32, order='F') / 32768.0
# what we are using 
arm_spct = get_arm_spectrogram(sound)
# what were not using 
tf_spect = tf.signal.stft(sound, frame_length=512, frame_step=128).numpy().astype(np.float32)
fig = px.imshow(np.array(arm_spct))
run.log({f'spectrogram_{category}':fig})

[   0    0    0 ... -591 -643 -631]


/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_41368/3992365391.py:8: ComplexWarning:

Casting complex values to real discards the imaginary part



In [89]:
run.finish()


In [23]:
IFrame('https://wandb.ai/tiny-ml/wake_word_detection/reports/yes-spectrograms--VmlldzozNDU0NDI3', 
       width=1900, 
       height=800)

In [24]:
IFrame('https://wandb.ai/tiny-ml/wake_word_detection/reports/Walking-Rubbix-Cube--VmlldzozNDU0NTI5', 
       width=1900, height=800)

# Adding spectrogram plot to wandb table 🌌

# logging some media outside of a table to workspace
This is for:
- cheching what our network see in signal domain;
- to spot patterns;
- gain intuitions about process and raw signal data.

In [19]:
run = wandb.init(id=run.id,
                 entity='tiny-ml',
                 project = 'wake_word_detection ',
                 group='Data',
                 resume='must')
sample_rate = 16000
category = 'yes'
n_samples = 10
for idx in tqdm(range(1,n_samples), colour=next(tqdm_colours)):
    sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
    sound = sound.astype(np.float32, order='F') / 32768.0
    arm_spect = get_arm_spectrogram(sound)
    img = plot_spectrogram(arm_spect,sample_rate=sample_rate)
    img = wandb.Image(img)
    run.log({f'spectrogram_image{category}':img})

 11%|███████████▉                                                                                               | 1/9 [00:00<00:07,  1.03it/s]Traceback (most recent call last):
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 112, in run
    shandler(sreq)
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 174, in server_record_publish
    iface = self._mux.get_stream(stream_id).interface
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/service/streams.py", line 206, in get_stream
    stream = self._streams[stream_id]
KeyError: 'mi552m39'
100%|███████████████████████████████████████████████████████████████████████████████████████████████████

In [25]:
IFrame('https://wandb.ai/tiny-ml/wake_word_detection/reports/WandB-Table-with-Sound-Spectrogram--VmlldzozNDU0NDc5', 
       width=1900, height=800)

In [20]:
run.finish()
run

In [64]:
# !rm -rf datasets
# artifact = wandb.Artifact(name='metadata', type='ESC-50-master')
# artifact.add_reference('https://github.com/karoldvl/ESC-50/archive/master.zip')
# artifact = run.use_artifact('tiny-ml/wake_word_detection/metadata:v0', type='ESC-50-master')
# artifact_dir = artifact.download()
# !mkdir datasets
# !unzip ./artifacts/metadata:v0/master.zip -d /Users/fridadesigley/pico/tiny-ml/datasets/ 

# 2. This Covers Pre Processing on ESC 50 dataset 💅🏻

### Reading our pre-traning data to pandas 🐼

In [21]:
run = wandb.init(id=run.id,
                 entity='tiny-ml',
                 project = 'wake_word_detection', 
                 group='Data',
                 resume='must')
esc50_csv = './datasets/ESC-50-master/meta/esc50.csv'
df = pd.read_csv(esc50_csv)
initial_data_table = wandb.Table(data=df, columns=list(df.columns))
run.log({'initial_table':initial_data_table})

Problem at: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_34488/1036862475.py 1 <module>


Traceback (most recent call last):
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 1108, in init
    run = wi.init()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 742, in init
    raise error
wandb.errors.CommError: Error communicating with wandb process, exiting...
For more info see: https://docs.wandb.ai/library/init#init-start-error


Exception: problem

### Here we are training on 1 second sound clips and our ESC 50 Dataset is 4 seconds per class

### testing our function 

In [ ]:
test_file = 'datasets/ESC-50-master/audio/1-100032-A-0.wav'
data = segment(fid=test_file,chunk=1)


### reading 1 second clips writing to a new files

In [ ]:
in_paranet = Path('./datasets/ESC-50-master/audio/')
out_data_dir = Path('ESC-50')
out_data_dir.mkdir(exist_ok=True)
all_out = [ ]
for idx, row in tqdm(df.iterrows()):
    out_dir = out_data_dir/row.category
    out_dir.mkdir(exist_ok=True)
    data, vals = segment(fid = str(in_paranet/row.filename),
       chunk=1)
    chans, samp_width , rate = vals
    out_fids = [ ]
    for idx,sound in enumerate(data):
        out_fid = out_dir/f'{idx}_{row.filename}'
        out_fids.append(out_fid)
        with wave.open(str(out_fid), 'w') as outfile:
            outfile.setnchannels(chans)
            outfile.setsampwidth(samp_width)
            outfile.setframerate(rate)
            outfile.setnframes(int(len(sound) /  samp_width))
            outfile.writeframes(sound)
    all_out.append(out_fids)


# Building an Artifact of 1 second sound clips broken down into class/category name
This:
> - creates a type which is here a parent directory
> - names an artfact for each class
> - creates a list of artfacts which are then itterated through to add a dirctory

#### Our artifacts are going to follow this pattern

In [78]:
import shutil
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')

esc50_csv = Path('./datasets/ESC-50-master/meta/esc50.csv')
path = Path('ESC-50/')
meta_data = path/'meta_data'
meta_data.mkdir(exist_ok=True)
shutil.copyfile(esc50_csv,meta_data/esc50_csv.name)
esc_artifacts = [pth for pth in path.iterdir()]
for pth in tqdm(esc_artifacts[-2:],**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=pth)
    
run.finish()


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  3.19it/s]


### adding our one second sound file paths to our data frame

In [79]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
df = pd.read_csv(esc50_csv)
out_esc50_csv = meta_data/esc50_csv.name
sec_files = np.array(all_out).astype(str)
for files in range(sec_files.shape[-1]):
    df[f'{files}_{files+1}_sec']=sec_files[...,files].astype(str)
df.to_csv(out_esc50_csv)
log_wandb_artifact(run=run,path=out_esc50_csv)
df

,filename,fold,target,category,esc10,src_file,take,0_1_sec,1_2_sec,2_3_sec,3_4_sec,4_5_sec,5_6_sec
0,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/0_1-100032-A-0.wav,ESC-50/dog/1_1-100032-A-0.wav,ESC-50/dog/2_1-100032-A-0.wav,ESC-50/dog/3_1-100032-A-0.wav,ESC-50/dog/4_1-100032-A-0.wav,ESC-50/dog/5_1-100032-A-0.wav
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,ESC-50/chirping_birds/0_1-100038-A-14.wav,ESC-50/chirping_birds/1_1-100038-A-14.wav,ESC-50/chirping_birds/2_1-100038-A-14.wav,ESC-50/chirping_birds/3_1-100038-A-14.wav,ESC-50/chirping_birds/4_1-100038-A-14.wav,ESC-50/chirping_birds/5_1-100038-A-14.wav
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,ESC-50/vacuum_cleaner/0_1-100210-A-36.wav,ESC-50/vacuum_cleaner/1_1-100210-A-36.wav,ESC-50/vacuum_cleaner/2_1-100210-A-36.wav,ESC-50/vacuum_cleaner/3_1-100210-A-36.wav,ESC-50/vacuum_cleaner/4_1-100210-A-36.wav,ESC-50/vacuum_cleaner/5_1-100210-A-36.wav
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,ESC-50/vacuum_cleaner/0_1-100210-B-36.wav,ESC-50/vacuum_cleaner/1_1-100210-B-36.wav,ESC-50/vacuum_cleaner/2_1-100210-B-36.wav,ESC-50/vacuum_cleaner/3_1-100210-B-36.wav,ESC-50/vacuum_cleaner/4_1-100210-B-36.wav,ESC-50/vacuum_cleaner/5_1-100210-B-36.wav
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,ESC-50/thunderstorm/0_1-101296-A-19.wav,ESC-50/thunderstorm/1_1-101296-A-19.wav,ESC-50/thunderstorm/2_1-101296-A-19.wav,ESC-50/thunderstorm/3_1-101296-A-19.wav,ESC-50/thunderstorm/4_1-101296-A-19.wav,ESC-50/thunderstorm/5_1-101296-A-19.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5-263831-B-6.wav,5,6,hen,False,263831,B,ESC-50/hen/0_5-263831-B-6.wav,ESC-50/hen/1_5-263831-B-6.wav,ESC-50/hen/2_5-263831-B-6.wav,ESC-50/hen/3_5-263831-B-6.wav,ESC-50/hen/4_5-263831-B-6.wav,ESC-50/hen/5_5-263831-B-6.wav
1996,5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A,ESC-50/vacuum_cleaner/0_5-263902-A-36.wav,ESC-50/vacuum_cleaner/1_5-263902-A-36.wav,ESC-50/vacuum_cleaner/2_5-263902-A-36.wav,ESC-50/vacuum_cleaner/3_5-263902-A-36.wav,ESC-50/vacuum_cleaner/4_5-263902-A-36.wav,ESC-50/vacuum_cleaner/5_5-263902-A-36.wav
1997,5-51149-A-25.wav,5,25,footsteps,False,51149,A,ESC-50/footsteps/0_5-51149-A-25.wav,ESC-50/footsteps/1_5-51149-A-25.wav,ESC-50/footsteps/2_5-51149-A-25.wav,ESC-50/footsteps/3_5-51149-A-25.wav,ESC-50/footsteps/4_5-51149-A-25.wav,ESC-50/footsteps/5_5-51149-A-25.wav
1998,5-61635-A-8.wav,5,8,sheep,False,61635,A,ESC-50/sheep/0_5-61635-A-8.wav,ESC-50/sheep/1_5-61635-A-8.wav,ESC-50/sheep/2_5-61635-A-8.wav,ESC-50/sheep/3_5-61635-A-8.wav,ESC-50/sheep/4_5-61635-A-8.wav,ESC-50/sheep/5_5-61635-A-8.wav


In [80]:
run

In [81]:
meta_data_table = wandb.Table(data=df,columns=list(df.columns))
run.log({'meta_data_without_media':meta_data_table})


# Data table for sound Wandb table with 

> 1. ouriginal sound string
> 2. playable sound file
> 3. 4 * 1 second clips files
> 4  4 * 1 second clips files

# Sound and Spectrogram Table



## Adding Sound and Spectrograms to WandB Table  🔉 🪄 🐝

In [18]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
n_examples = 2
n_samples = 10
s_df = df.sample(n=n_samples, random_state=2)
columns = list(df.columns)
table_with_media = wandb.Table(data=s_df,columns=columns)
for colmn in df.columns[-n_examples:]:
    # create wandb sound objects
    sounds = [wandb.Audio(fid) for fid in tqdm(s_df[colmn].values,**tqdm_args,desc=colmn)]
    # add these as a column to our wandb table
    table_with_media.add_column(name=f'sound_{colmn}',data=sounds)

NameError: name 'df' is not defined

### Adding Spectrograms to wandb Table

In [19]:
for col in df.columns[-n_examples:]:
    # read wavs 
    wavs = map(read_wav,tqdm(s_df[col].values,**tqdm_args,desc=col))
    # generate spectrograms
    spects = map(get_arm_spectrogram,wavs)
    sample_rates = iter(np.full((1,len(df)),1600)[0])
    # get numpy arrays of save .jpg files
    ims = list(map(plot_spectrogram,spects,sample_rates))
    # create a list of wandb images
    spects = list(map(wandb.Image,tqdm(ims)))
    # add these to a table 
    table_with_media.add_column(name=f'image_{col}',data=spects)
run.log({'sound_spectrogram_table':table_with_media})

NameError: name 'df' is not defined

## Stack our pandas df by 1 second Files 🧱

In [84]:
rp_df = pd.DataFrame(np.repeat(df.values, 6, axis=0))
rp_df.columns=df.columns
a,b,c,d,e,f = np.array([df[col].values for col in df.columns[-6:]])
all_files =np.vstack((a,b,c,d,e,f)).reshape((-1,),order='F')
rp_df = rp_df[df.columns[:-6]]
rp_df['all_files']=all_files
df = rp_df  
df.to_csv(out_esc50_csv)
log_wandb_artifact(run,out_esc50_csv)
df

,filename,fold,target,category,esc10,src_file,take,all_files
0,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/0_1-100032-A-0.wav
1,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/1_1-100032-A-0.wav
2,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/2_1-100032-A-0.wav
3,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/3_1-100032-A-0.wav
4,1-100032-A-0.wav,1,0,dog,True,100032,A,ESC-50/dog/4_1-100032-A-0.wav
...,...,...,...,...,...,...,...,...
11995,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/1_5-9032-A-0.wav
11996,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/2_5-9032-A-0.wav
11997,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/3_5-9032-A-0.wav
11998,5-9032-A-0.wav,5,0,dog,True,9032,A,ESC-50/dog/4_5-9032-A-0.wav


### Define Test,Train, Val using pandas & add sets column to our dataset

In [86]:
sets = { }
train_val = df.sample(frac=0.9,random_state=42)
sets['test']=df.drop(train_val.index)
sets['train']=train_val.sample(frac=0.9,random_state=42)
sets['val']=train_val.drop(sets['train'].index)
for name,subset in sets.items():
        subset = subset.reset_index(drop=True)
        subset['set']=[name for va in range(len(subset))]
        sets[name]=subset
df.groupby("target", group_keys=False).apply(lambda x: x)
df = pd.concat(sets.values()).reset_index(drop=True)
df.to_csv(out_esc50_csv)
log_wandb_artifact(run,out_esc50_csv)

In [92]:
run

# remove silence by randomly sampling from non silent

In [88]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
is_sound = [not np.std(read_wav(x)) < 0.1 for x in df.all_files.values]
df['is_sound']=is_sound
df.to_csv(out_esc50_csv)
log_wandb_artifact(run,out_esc50_csv)

In [89]:
silence = df[df.is_sound==False]
final_meta_table = wandb.Table(data=silence,columns=list(df.columns))
wandb_audio = list(map(wandb.Audio,silence.all_files.values))
final_meta_table.add_column(name='audio',data=wandb_audio)
run.log({'silence_tabel':final_meta_table})

In [93]:
run

# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [91]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)

file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


In [95]:
for pth in df[df.is_sound==False].all_files.values:
    path = Path(pth)
    try:
        path.unlink()
    except FileNotFoundError:
        print('file alredady deleted')
for path in tqdm(esc_artifacts,**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=path)

100%|███████████████████████████████████████████| 51/51 [00:16<00:00,  3.09it/s]


# Save 3D arrays of Spectrograms using numpy

In [96]:
def save_npz(x,y,name,augmenter=None):
    wavs = list(map(read_wav,tqdm(x,f'creating {name}')))
    if augmenter:
        samples = np.full((1,len(y)), 16000)[0]
        wavs = list(map(augmenter,wavs,tqdm(samples,f'augmenting {name}')))
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} augspects'))))
    else:
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} spects'))))
    for array in x_data:
        assert array.shape==(682, 257)
    np.savez(f'{name}.npz',x_data=x_data,y_data=y)

## Loop over files and save as npz, this is our pre-processed training data.

In [97]:
contains_sound = df[df.is_sound==True]
subset =  contains_sound[contains_sound.set=='train']


In [98]:
pre_process=True
contains_sound = df[df.is_sound==True]
if pre_process:
    for name in df.set.unique():
        subset =  contains_sound[contains_sound.set==name]
        x_data = subset['all_files'].values
        y_data = subset['target'].values
        save_npz(x_data,y_data,f'processed/{name}')
        if name=='train':
            print('augmenting')
            save_npz(x_data[:len(x_data)//5],y_data[:len(y_data)//5],f'processed/{name}_aug',augmenter=augmenter)

creating processed/train spects: 100%|█████| 9194/9194 [01:24<00:00, 108.78it/s]


augmenting


augmenting processed/train_aug: 100%|██████▉| 1837/1838 [03:45<00:00,  8.16it/s]
creating processed/train_aug augspects: 100%|█| 1838/1838 [00:15<00:00, 120.38it
creating processed/val spects: 100%|███████| 1030/1030 [00:08<00:00, 119.39it/s]


# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [99]:
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)

file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


### To finish our data run

In [100]:
run.finish()
run

# Part 3. Training

### Training load training dataset which has already been pre processed

In [90]:
import tensorflow as tf
import wandb
from pathlib import Path
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import numpy as np
import sounddevice as sd
import wave
from pvrecorder import PvRecorder
import time 
from utils.data_processing import log_wandb_artifact,plot_spectrogram,segment,read_wav,Arm_spect,augmenter
from utils.create_sweep import create_sweep

In [2]:
with np.load('processed/train.npz',allow_pickle=True) as data:
    train_x = data['x_data'].astype(np.float32)
    train_y = data['y_data'].astype(np.uint8)
with np.load('processed/train_aug.npz',allow_pickle=True) as data:
    aug_x = data['x_data'].astype(np.float32)
    aug_y = data['y_data'].astype(np.uint8)
with np.load('processed/val.npz',allow_pickle=True) as data:
    val_x = data['x_data'].astype(np.float32)
    val_y = data['y_data'].astype(np.uint8)
    

## create TensoFlow Dataset from numpy arrays of spectrograms and target variable

In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(train_x, tf.float32), train_y))
train_aug_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(aug_x, tf.float32), aug_y))
train_dataset = train_dataset.concatenate(train_aug_dataset)
val_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(val_x, tf.float32),val_y))
train_ds = train_dataset.cache().shuffle(100, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [4]:
x, y = next(iter(train_ds))
x[0],y[0]

2023-01-31 14:29:12.393101: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


(<tf.Tensor: shape=(682, 257), dtype=float32, numpy=
 array([[ 7.65625 ,  6.71875 , 22.296875, ...,  1.875   ,  2.765625,
          3.328125],
        [12.28125 , 10.046875, 21.640625, ...,  2.921875,  3.046875,
          2.5625  ],
        [ 9.      , 19.953125, 30.546875, ...,  2.75    ,  3.03125 ,
          0.796875],
        ...,
        [ 8.171875, 10.484375, 25.4375  , ...,  1.125   ,  0.875   ,
          0.421875],
        [ 1.765625, 18.71875 , 24.359375, ...,  1.25    ,  1.890625,
          0.703125],
        [ 6.4375  , 25.96875 , 31.125   , ...,  2.40625 ,  3.171875,
          2.390625]], dtype=float32)>,
 <tf.Tensor: shape=(), dtype=uint8, numpy=28>)

## Test that this dataset is loading okay

In [5]:
x,y = next(iter(train_dataset.take(1)))
#print(f' target = {y}, \n spectrogram = \n {x}')
input_shape = tf.expand_dims(x, axis=-1).shape
#print(input_shape)
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_dataset.map(lambda x, y: tf.reshape(x, input_shape)))
baseline_model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
  norm_layer,
  tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(50, activation='softmax')
])

# Create a new training run for our training 🏃🏻‍♀️ ♀️ 

In [87]:
run = wandb.init(entity='tiny-ml',project = 'wake_word_detection', group='training')

## Define our model 🏗️

In [89]:
baseline_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_3 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_1 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 8)         520       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 288)               0         
                                                                 
 dropout_3 (Dropout)         (None, 288)              

In [90]:
METRICS = ["accuracy",]
baseline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=METRICS,
)

In [91]:

# Initialize a new W&B run 
checkpoint_path = "training_1/"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights

In [94]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
    tf.keras.callbacks.LearningRateScheduler(scheduler)
,cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)]

In [95]:
run

In [96]:
EPOCHS = 50
history = baseline_model.fit(
     train_ds, 
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=val_ds
)

Epoch 1/50


ValueError: in user code:

    File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        rais

In [97]:
run.finish()

## WandB Sweep 🧹

In [54]:
IFrame('https://wandb.ai/tiny-ml/wake_word_detection/reports/Sweep-Only--VmlldzozNDUzNjIy', 
       width=1900,  
       height=850)

In [92]:
from utils import create_sweep
from utils.pre_train_sweep import pre_train
import wandb
import os
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [57]:
entity, project ='tiny-ml', 'wake_word_detection'
sweep_id = create_sweep(**{'project':project,'entity':entity})
wandb.agent(sweep_id, function=pre_train,count=1 )

Create sweep with ID: fjitorla
Sweep URL: https://wandb.ai/tiny-ml/wake_word_detection/sweeps/fjitorla
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-01-31 17:32:16.089065: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100
17/21 [=======================>......] - ETA: 0s - loss: 3.9837 - accuracy: 0.0325
Epoch 1: saving model to training_1/
21/21 [==============================] - 1s 34ms/step - loss: 3.9734 - accuracy: 0.0350 - val_loss: 3.9091 - val_accuracy: 0.0231 - lr: 7.0000e-04
Epoch 2/100
16/21 [=====================>........] - ETA: 0s - loss: 3.8968 - accuracy: 0.0384
Epoch 2: saving model to training_1/
21/21 [==============================] - 0s 16ms/step - loss: 3.8984 - accuracy: 0.0415 - val_loss: 3.8554 - val_accuracy: 0.0417 - lr: 7.0000e-04
Epoch 3/100
18/21 [========================>.....] - ETA: 0s - loss: 3.8261 - accuracy: 0.0538
Epoch 3: saving model to training_1/
21/21 [==============================] - 0s 18ms/step - loss: 3.8216 - accuracy: 0.0555 - val_loss: 3.7875 - val_accuracy: 0.0583 - lr: 7.0000e-04
Epoch 4/100
16/21 [=====================>........] - ETA: 0s - loss: 3.7410 - accuracy: 0.0671
Epoch 4: saving model to training_1/
21/21 [========================

18/21 [========================>.....] - ETA: 0s - loss: 3.1094 - accuracy: 0.2060
Epoch 30: saving model to training_1/
21/21 [==============================] - 0s 14ms/step - loss: 3.1227 - accuracy: 0.2030 - val_loss: 3.3130 - val_accuracy: 0.1463 - lr: 9.4735e-05
Epoch 31/100
16/21 [=====================>........] - ETA: 0s - loss: 3.1420 - accuracy: 0.2038
Epoch 31: saving model to training_1/
21/21 [==============================] - 0s 14ms/step - loss: 3.1402 - accuracy: 0.2040 - val_loss: 3.3109 - val_accuracy: 0.1454 - lr: 8.5720e-05
Epoch 32/100
21/21 [==============================] - ETA: 0s - loss: 3.1392 - accuracy: 0.1970
Epoch 32: saving model to training_1/
21/21 [==============================] - 0s 15ms/step - loss: 3.1392 - accuracy: 0.1970 - val_loss: 3.3093 - val_accuracy: 0.1481 - lr: 7.7562e-05
Epoch 33/100
19/21 [==========================>...] - ETA: 0s - loss: 3.1428 - accuracy: 0.1919
Epoch 33: saving model to training_1/
21/21 [=============================

18/21 [========================>.....] - ETA: 0s - loss: 3.1299 - accuracy: 0.2014
Epoch 59: saving model to training_1/
21/21 [==============================] - 0s 14ms/step - loss: 3.1240 - accuracy: 0.1995 - val_loss: 3.2950 - val_accuracy: 0.1519 - lr: 5.2126e-06
Epoch 60/100
18/21 [========================>.....] - ETA: 0s - loss: 3.0897 - accuracy: 0.2072
Epoch 60: saving model to training_1/
21/21 [==============================] - 0s 13ms/step - loss: 3.1057 - accuracy: 0.2030 - val_loss: 3.2948 - val_accuracy: 0.1509 - lr: 4.7166e-06
Epoch 61/100
21/21 [==============================] - ETA: 0s - loss: 3.1135 - accuracy: 0.2060
Epoch 61: saving model to training_1/
21/21 [==============================] - 0s 12ms/step - loss: 3.1135 - accuracy: 0.2060 - val_loss: 3.2948 - val_accuracy: 0.1519 - lr: 4.2677e-06
Epoch 62/100
20/21 [===========================>..] - ETA: 0s - loss: 3.1264 - accuracy: 0.2094
Epoch 62: saving model to training_1/
21/21 [=============================

19/21 [==========================>...] - ETA: 0s - loss: 3.1347 - accuracy: 0.2007
Epoch 88: saving model to training_1/
21/21 [==============================] - 0s 13ms/step - loss: 3.1202 - accuracy: 0.2010 - val_loss: 3.2941 - val_accuracy: 0.1509 - lr: 2.8681e-07
Epoch 89/100
15/21 [====================>.........] - ETA: 0s - loss: 3.0971 - accuracy: 0.2035
Epoch 89: saving model to training_1/
21/21 [==============================] - 0s 12ms/step - loss: 3.0979 - accuracy: 0.1975 - val_loss: 3.2941 - val_accuracy: 0.1509 - lr: 2.5952e-07
Epoch 90/100
19/21 [==========================>...] - ETA: 0s - loss: 3.1093 - accuracy: 0.1946
Epoch 90: saving model to training_1/
21/21 [==============================] - 0s 12ms/step - loss: 3.1195 - accuracy: 0.1900 - val_loss: 3.2941 - val_accuracy: 0.1509 - lr: 2.3482e-07
Epoch 91/100
21/21 [==============================] - ETA: 0s - loss: 3.1079 - accuracy: 0.2005
Epoch 91: saving model to training_1/
21/21 [=============================

### Evaluate our Pre-trained Model

In [93]:
with np.load('processed/test.npz',allow_pickle=True) as data:
    test_x = data['x_data'].astype(np.float32)
    test_y = data['y_data'].astype(np.uint8)
test_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(test_x,tf.float32),test_y))
test_ds = test_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)
x,y = next(iter(test_dataset.take(1)))


In [94]:
input_shape = tf.expand_dims(x, axis=-1).shape
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(test_dataset.map(lambda x, y: tf.reshape(x, input_shape)))
baseline_model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
  norm_layer,
  tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(50, activation='softmax')
])
METRICS = ["accuracy",]

In [95]:
baseline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=METRICS,
)
baseline_model.evaluate(test_ds, verbose=2)

36/36 - 0s - loss: 4.0361 - accuracy: 0.0209 - 420ms/epoch - 12ms/step


[4.036131858825684, 0.02088772878050804]

### Dowload model artifact/ our registered model  🛰️📡 

In [97]:
api = wandb.Api()
id_ = 'ex78039g'
artifact = api.artifact(f'tiny-ml/wake_word_detection/run_{id_}_model:latest', type='model')
print(artifact.digest,artifact.aliases)
file = artifact.download()  

fb60627ca17c932d33d95875ef1c3b14 ['latest', 'epoch_50']


### Load our pre trained tiny model from wandb artifact 🏺🏺🏺

In [98]:
baseline_model.load_weights(f'{file}/cp.ckpt')
baseline_model.evaluate(test_ds, verbose=2)

36/36 - 0s - loss: 2.4874 - accuracy: 0.3211 - 83ms/epoch - 2ms/step


[2.487403392791748, 0.3211488127708435]

### Create our wake work training dataset, this time its a lot smaller.

In [32]:
sample_rate = 16000
get_arm_spectrogram = Arm_spect().get_arm_spectrogram
path = Path('./data/') 
paths = {pth.name:list(map(str,filter(lambda x:x.name != '.gitkeep',pth.iterdir()))) 
         for pth in path.iterdir() if pth.name in ['background','yes']}
paths = {k:np.stack(list(filter(lambda x:len(x)==16384,map(read_wav,tqdm(v))))) 
         for (k,v) in tqdm(paths.items())}
for k,v in paths.items():
    filler_x = [augmenter(x,sample_rate) 
                for x in v[np.random.choice(v.shape[0], 500-len(v), replace=True)]]
    filler_x = np.stack(list(map(lambda x:x[:16384], tqdm(filler_x))))
    paths[k]=np.array(list(map(get_arm_spectrogram,tqdm(np.vstack([v,filler_x]))))).astype(np.float32)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 2659.41it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 385.06it/s]


In [99]:
y = np.concatenate([np.zeros((1,500)).astype(np.uint16)[0], np.ones((1,500)).astype(np.uint16)[0]])
x = np.vstack(list(paths.values()))

In [102]:
train_wake_data = tf.data.Dataset.from_tensor_slices((tf.cast(x,tf.float32), y))
train_wake_loader = train_wake_data.cache().shuffle(2000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)

In [103]:
x,y = next(iter(train_wake_data.take(1)))
input_shape = tf.expand_dims(x, axis=-1).shape
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_wake_data.map(lambda x, y: tf.reshape(x, input_shape)))


In [104]:
input_shape = tf.expand_dims(x, axis=-1).shape
print(input_shape)

(249, 257, 1)


In [105]:
base_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
    norm_layer,
    tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(50, activation='softmax')
    ])


In [107]:
model_body = tf.keras.Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
classifier_head = tf.keras.layers.Dense(1, activation="sigmoid")(model_body.output)
wake_word_model = tf.keras.Model(model_body.input, classifier_head)

wake_word_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=METRICS,
)


In [108]:
def scheduler(epoch, lr):
    ''' a function to increase lr at start of trining
    '''
    if epoch < 10:
        return lr
    else:
        # add somthing like np.linespace([0,-0.1])
        return lr * tf.math.exp(-0.1)

In [109]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [110]:
run = wandb.init(entity='tiny-ml',project = 'wake_word_detection', group='training')
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)
]

In [111]:
checkpoint_path = "training_2/"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [112]:
wake_word_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 249, 257, 1)]     0         
                                                                 
 resizing_5 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_5 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 8)         520       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 288)               0   

### Freeeeez Layers ❄️❄️❄️

In [113]:
for layer in wake_word_model.layers:
    layer.trainable = False
wake_word_model.layers[-1].trainable = True

## Compile model 🦿🤖🦾

In [114]:
wake_word_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=METRICS,
)

### Train on Yes vs Background 

In [115]:
EPOCHS = 10
history = wake_word_model.fit(
    train_wake_loader, 
    epochs=EPOCHS,
    callbacks=callbacks
)
run.finish()

Epoch 1/10
26/32 [=======================>......] - ETA: 0s - loss: 0.6751 - accuracy: 0.6238WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy



Epoch 1: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets


32/32 [==============================] - 1s 38ms/step - loss: 0.6714 - accuracy: 0.6320 - lr: 0.0030
Epoch 2/10
23/32 [====================>.........] - ETA: 0s - loss: 0.6347 - accuracy: 0.6807WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


...

32/32 [==============================] - 2s 52ms/step - loss: 0.5335 - accuracy: 0.7510 - lr: 0.0030
Epoch 10/10
23/32 [====================>.........] - ETA: 0s - loss: 0.5253 - accuracy: 0.7459WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy



Epoch 10: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets


32/32 [==============================] - 1s 30ms/step - loss: 0.5295 - accuracy: 0.7490 - lr: 0.0030


In [117]:
run

## Unfreeze layers 🥶

In [118]:
for layer in wake_word_model.layers:
    layer.trainable = True

In [119]:
import tensorflow_model_optimization as tfmot
def apply_qat_to_dense_and_cnn(layer):
  if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer
annotated_model = tf.keras.models.clone_model(
    wake_word_model,
    clone_function=apply_qat_to_dense_and_cnn,
)
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 249, 257, 1)]     0         
                                                                 
 resizing_5 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_5 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 quant_conv2d_5 (QuantizeWra  (None, 13, 13, 8)        539       
 pperV2)                                                         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                           

In [121]:
quant_aware_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=METRICS,
)
EPOCHS=50
quant_aware_history = quant_aware_model.fit(
    train_wake_loader,  
    epochs=EPOCHS
)

Epoch 1/50
32/32 [==============================] - 1s 3ms/step - loss: 0.3330 - accuracy: 0.8610
Epoch 2/50
32/32 [==============================] - 0s 3ms/step - loss: 0.3391 - accuracy: 0.8630
Epoch 3/50
32/32 [==============================] - 0s 3ms/step - loss: 0.3124 - accuracy: 0.8710
Epoch 4/50
32/32 [==============================] - 0s 3ms/step - loss: 0.3001 - accuracy: 0.8740
Epoch 5/50
32/32 [==============================] - 0s 3ms/step - loss: 0.3082 - accuracy: 0.8700
Epoch 6/50
32/32 [==============================] - 0s 4ms/step - loss: 0.3099 - accuracy: 0.8650
Epoch 7/50
32/32 [==============================] - 0s 3ms/step - loss: 0.2935 - accuracy: 0.8860
Epoch 8/50
32/32 [==============================] - 0s 3ms/step - loss: 0.2700 - accuracy: 0.8930
Epoch 9/50
32/32 [==============================] - 0s 3ms/step - loss: 0.2768 - accuracy: 0.8880
Epoch 10/50
32/32 [==============================] - 0s 3ms/step - loss: 0.2730 - accuracy: 0.8930
Epoch 11/50
32/32 [

In [122]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert() 

INFO:tensorflow:Assets written to: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmpktqilq48/assets


INFO:tensorflow:Assets written to: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmpktqilq48/assets
/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:765: UserWarning:

Statistics for quantized inputs were expected, but not specified; continuing anyway.

2023-01-31 18:38:36.910326: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-31 18:38:36.910337: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-31 18:38:36.910414: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmpktqilq48
2023-01-31 18:38:36.911689: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-31 18:38:36.911698: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/q8/9dhs6y4d21s87

In [124]:
tflite_models_dir = Path("tf_lite_model")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/"wake_word.tflite"
tflite_model_file.write_bytes(tflite_model_quant) 

6016

In [125]:
%%bash
echo "alignas(8) const unsigned char tflite_model[] = {" > tf_lite_model/wake_word.tflite.h
cat tf_lite_model/wake_word.tflite | xxd -i                        >> tf_lite_model/wake_word.tflite.h
echo "};"                                               >> tf_lite_model/wake_word.tflite.h

In [126]:
!ls tf_lite_model/

wake_word.tflite   wake_word.tflite.h


In [127]:
!cat tf_lite_model/wake_word.tflite.h

alignas(8) const unsigned char tflite_model[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x88, 0x00, 0x00, 0x00, 0xe0, 0x00, 0x00, 0x00,
  0x74, 0x0b, 0x00, 0x00, 0x84, 0x0b, 0x00, 0x00, 0x98, 0x16, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x46, 0xf3, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x40, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x94, 0xff, 0xff, 0xff,
  0x0e, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00,
  0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x64, 0x65, 0x6e, 0x73, 0x65, 0x5f,
  0x39, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 

```bash
alignas(8) const unsigned char tflite_model[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x88, 0x00, 0x00, 0x00, 0xe0, 0x00, 0x00, 0x00,
  0x74, 0x0b, 0x00, 0x00, 0x84, 0x0b, 0x00, 0x00, 0x98, 0x16, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x46, 0xf3, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x40, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x94, 0xff, 0xff, 0xff,
  0x0e, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00,
  0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x64, 0x65, 0x6e, 0x73, 0x65, 0x5f,
  0x33, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x12, 0xfd, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00,
  0x69, 0x6e, 0x70, 0x75, 0x74, 0x5f, 0x33, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xdc, 0xff, 0xff, 0xff,
  0x11, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00,
  0x43, 0x4f, 0x4e, 0x56, 0x45, 0x52, 0x53, 0x49, 0x4f, 0x4e, 0x5f, 0x4d,
  0x45, 0x54, 0x41, 0x44, 0x41, 0x54, 0x41, 0x00, 0x08, 0x00, 0x0c, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x08, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f,
  0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73,
  0x69, 0x6f, 0x6e, 0x00, 0x12, 0x00, 0x00, 0x00, 0x90, 0x0a, 0x00, 0x00,
  0x88, 0x0a, 0x00, 0x00, 0x78, 0x02, 0x00, 0x00, 0x58, 0x02, 0x00, 0x00,
  0x40, 0x02, 0x00, 0x00, 0x10, 0x02, 0x00, 0x00, 0xfc, 0x01, 0x00, 0x00,
  0xf4, 0x01, 0x00, 0x00, 0xec, 0x01, 0x00, 0x00, 0xe4, 0x01, 0x00, 0x00,
  0xdc, 0x01, 0x00, 0x00, 0xd4, 0x01, 0x00, 0x00, 0xa4, 0x00, 0x00, 0x00,
  0x9c, 0x00, 0x00, 0x00, 0x94, 0x00, 0x00, 0x00, 0x8c, 0x00, 0x00, 0x00,
  0x6c, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xce, 0xfd, 0xff, 0xff,
  0x04, 0x00, 0x00, 0x00, 0x58, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x0e, 0x00, 0x08, 0x00, 0x04, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x10, 0x00, 0x00, 0x00, 0x28, 0x00, 0x00, 0x00, 0x00, 0x00, 0x06, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x06, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0xd0, 0x07, 0x00, 0x00, 0x00, 0x00, 0x0a, 0x00,
  0x10, 0x00, 0x0c, 0x00, 0x08, 0x00, 0x04, 0x00, 0x0a, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x06, 0x00, 0x00, 0x00, 0x32, 0x2e, 0x31, 0x31, 0x2e, 0x30, 0x00, 0x00,
  0x32, 0xfe, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x32, 0x2e, 0x33, 0x2e, 0x30, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x44, 0xeb, 0xff, 0xff, 0x48, 0xeb, 0xff, 0xff,
  0x4c, 0xeb, 0xff, 0xff, 0x5a, 0xfe, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00,
  0x20, 0x01, 0x00, 0x00, 0xf0, 0x3a, 0x3c, 0x1d, 0x17, 0xfa, 0xf6, 0x0d,
  0xf3, 0xfc, 0xec, 0xd9, 0xfe, 0x01, 0xc5, 0xcb, 0x04, 0xd8, 0xe0, 0x08,
  0xd6, 0x4e, 0xd1, 0xf1, 0xe9, 0xfd, 0x10, 0xe1, 0xef, 0x01, 0xd2, 0xef,
  0xfd, 0xdd, 0x1d, 0xd3, 0x61, 0x06, 0x0b, 0x31, 0x21, 0x43, 0x34, 0x09,
  0x44, 0x33, 0x27, 0x29, 0xed, 0x24, 0x37, 0xf8, 0x25, 0x16, 0xfb, 0x10,
  0xfa, 0xfd, 0xf5, 0xda, 0xe1, 0x14, 0xd0, 0xea, 0x0e, 0xda, 0x11, 0xb5,
  0x1b, 0x0a, 0xce, 0x20, 0xf4, 0xeb, 0x13, 0xc4, 0x16, 0xfe, 0xc2, 0x02,
  0xee, 0x12, 0x14, 0xbf, 0x26, 0x53, 0x28, 0x13, 0x3b, 0x27, 0x30, 0xed,
  0x7f, 0x66, 0x69, 0x49, 0xf8, 0x1f, 0x13, 0x06, 0x19, 0x1e, 0xee, 0xe5,
  0xf2, 0x11, 0xe8, 0xf2, 0xde, 0x08, 0xbc, 0xf6, 0xce, 0x13, 0x08, 0xc3,
  0x20, 0xf9, 0xcf, 0xda, 0x04, 0xbb, 0xfa, 0xce, 0x00, 0x1a, 0xc9, 0xda,
  0x31, 0xc9, 0x0e, 0xe2, 0x11, 0x5e, 0x0a, 0x02, 0x07, 0xfb, 0x2f, 0xe1,
  0x44, 0x7f, 0x28, 0x0c, 0xee, 0x15, 0x16, 0x24, 0x00, 0x20, 0x0b, 0xe1,
  0xe1, 0x9c, 0xe5, 0x14, 0xfd, 0x21, 0xc6, 0xe6, 0xe0, 0xf9, 0x13, 0xf1,
  0xee, 0xd5, 0xd9, 0xc1, 0x0b, 0xeb, 0x0a, 0xdf, 0x1b, 0x00, 0xd4, 0xdb,
  0x01, 0x34, 0x2e, 0xe0, 0x40, 0x29, 0x30, 0x02, 0x2e, 0x18, 0x17, 0xf3,
  0x5a, 0x4d, 0x22, 0x1d, 0xec, 0x45, 0x24, 0xf8, 0x01, 0x2e, 0xd6, 0xfe,
  0xe0, 0xe8, 0xda, 0xda, 0x04, 0x08, 0xac, 0x03, 0xf5, 0xae, 0xef, 0xe7,
  0xec, 0x07, 0xbc, 0xf2, 0xd6, 0xf6, 0x11, 0xc5, 0x04, 0x06, 0xe1, 0xc2,
  0x2c, 0xfa, 0x35, 0xcf, 0x31, 0x3d, 0x07, 0x0d, 0x28, 0xe3, 0x51, 0xf2,
  0x45, 0x21, 0x39, 0x16, 0xf5, 0x11, 0x0b, 0x0d, 0x0d, 0x04, 0xc8, 0x15,
  0xf8, 0x26, 0xbd, 0x2a, 0xeb, 0xf2, 0xc7, 0x21, 0xf3, 0x22, 0xe0, 0xf2,
  0xfd, 0xf4, 0xc0, 0xfc, 0xd5, 0xf4, 0xfb, 0xfb, 0xf7, 0x27, 0xd5, 0xcb,
  0x03, 0xf7, 0x22, 0xef, 0x3d, 0x2e, 0x1f, 0x28, 0x4e, 0x0c, 0x37, 0xfd,
  0x38, 0x07, 0x1c, 0x3c, 0x7c, 0xec, 0xff, 0xff, 0x80, 0xec, 0xff, 0xff,
  0x84, 0xec, 0xff, 0xff, 0x88, 0xec, 0xff, 0xff, 0x8c, 0xec, 0xff, 0xff,
  0x9a, 0xff, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x76, 0xfc, 0xff, 0xff, 0xaa, 0xff, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0x37, 0xb9, 0x37, 0x3d, 0xc1, 0xc5, 0x1d, 0x3c,
  0x7c, 0x44, 0xac, 0xbe, 0x77, 0x96, 0x48, 0x3e, 0xc7, 0x84, 0xb8, 0xbe,
  0x0b, 0x9e, 0xa1, 0xbe, 0x74, 0xdd, 0xac, 0x3a, 0x4d, 0xc3, 0xbf, 0x3c,
  0xd6, 0xff, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0xff, 0xff, 0xff, 0xff, 0x20, 0x01, 0x00, 0x00, 0xea, 0xff, 0xff, 0xff,
  0x04, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0x00, 0x00, 0x06, 0x00, 0x08, 0x00, 0x04, 0x00,
  0x06, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00,
  0x96, 0x80, 0x1d, 0xbe, 0x1f, 0x76, 0xee, 0x3d, 0x49, 0x4d, 0xc2, 0x3d,
  0xdb, 0x88, 0x01, 0x3d, 0xb2, 0xb1, 0xad, 0x3d, 0xaf, 0x01, 0x03, 0xbe,
  0x38, 0xf7, 0xd3, 0xbd, 0xba, 0xdc, 0xa4, 0xbc, 0x27, 0xa1, 0x65, 0xbd,
  0x90, 0x05, 0xd1, 0x3d, 0xd3, 0x5d, 0x8a, 0x3d, 0xba, 0xdc, 0x24, 0x3c,
  0x99, 0x30, 0x48, 0xbd, 0xcb, 0x32, 0x13, 0xbd, 0x1a, 0x16, 0x19, 0x3d,
  0xba, 0xdc, 0x24, 0xbc, 0x38, 0xf7, 0x53, 0x3d, 0x23, 0x41, 0x90, 0x3d,
  0x7b, 0x4f, 0x8d, 0x3c, 0x38, 0xf7, 0xd3, 0xbc, 0xf9, 0x69, 0xbc, 0x3d,
  0x01, 0x95, 0xb3, 0xbd, 0x27, 0xa1, 0x65, 0xbd, 0xd8, 0xbd, 0xdf, 0xbc,
  0xdb, 0x88, 0x01, 0xbd, 0x77, 0x84, 0x6b, 0xbd, 0xf9, 0x69, 0xbc, 0xba,
  0x27, 0xa1, 0x65, 0xbd, 0x17, 0x4b, 0xf7, 0xbc, 0x83, 0x7a, 0x04, 0xbe,
  0x0a, 0xc0, 0x2a, 0xbd, 0xcb, 0x32, 0x13, 0x3d, 0xdb, 0x88, 0x01, 0xbe,
  0xd8, 0xbd, 0xdf, 0x3c, 0x59, 0xa3, 0xb0, 0x3d, 0xd8, 0xbd, 0x5f, 0xbd,
  0x0a, 0xc0, 0xaa, 0xbd, 0x59, 0xa3, 0x30, 0x3d, 0x77, 0x84, 0xeb, 0xbb,
  0xcf, 0x92, 0xe8, 0xbd, 0x2b, 0x6c, 0x87, 0xbd, 0x01, 0x95, 0xb3, 0x3d,
  0xee, 0x8e, 0x1a, 0x3e, 0x38, 0xf7, 0xd3, 0x3d, 0xcb, 0x32, 0x13, 0xbd,
  0xc2, 0x07, 0x1c, 0xbe, 0x1f, 0x76, 0xee, 0x3d, 0xd8, 0xbd, 0xdf, 0xbc,
  0x99, 0x30, 0x48, 0xbd, 0x1a, 0x16, 0x19, 0x3d, 0x0a, 0xc0, 0x2a, 0x3d,
  0x4f, 0xc8, 0x0e, 0x3e, 0x17, 0x4b, 0xf7, 0xbc, 0x49, 0x4d, 0xc2, 0xbd,
  0x40, 0x22, 0xcb, 0xbd, 0x0a, 0xc0, 0x2a, 0x3d, 0x1a, 0x16, 0x99, 0xbc,
  0xf9, 0x69, 0x3c, 0xbc, 0x7b, 0x4f, 0x0d, 0x3c, 0x25, 0xf1, 0x3a, 0x3e,
  0x59, 0xa3, 0xb0, 0xbc, 0xa9, 0x86, 0x36, 0xbd, 0x49, 0x4d, 0xc2, 0x3d,
  0x90, 0x05, 0xd1, 0xbd, 0x2a, 0xf1, 0x10, 0xbe, 0xd3, 0x42, 0xa5, 0xbd,
  0x20, 0xaa, 0xfe, 0x3d, 0x5c, 0xbc, 0x58, 0x3d, 0xf1, 0x4b, 0x12, 0x3d,
  0x0d, 0xb7, 0x97, 0xbc, 0x7d, 0x63, 0xad, 0xbc, 0x46, 0x5c, 0x16, 0x3e,
  0x61, 0xf8, 0xa7, 0x3d, 0xd6, 0xe0, 0x8c, 0xbc, 0xb5, 0x39, 0xb8, 0x3d,
  0x96, 0x30, 0xcb, 0xbd, 0x7e, 0x32, 0x21, 0xbe, 0x20, 0xaa, 0x7e, 0xbd,
  0xd1, 0xa4, 0x3d, 0x3d, 0xcc, 0x68, 0x6e, 0x3c, 0x45, 0x8d, 0xa2, 0x3c,
  0x45, 0x8d, 0xa2, 0x3c, 0xf1, 0x4b, 0x92, 0xbd, 0x9e, 0x0a, 0x82, 0x3b,
  0x9e, 0x0a, 0x82, 0x3c, 0x06, 0xdd, 0xe0, 0x3d, 0xb0, 0xfd, 0x68, 0xbd,
  0xe8, 0xd3, 0xf3, 0xbd, 0x5c, 0xbc, 0x58, 0x3d, 0xd6, 0xe0, 0x8c, 0x3c,
  0x62, 0xc7, 0x1b, 0xbe, 0x78, 0x27, 0x5e, 0x3d, 0xed, 0x0f, 0x43, 0x3c,
  0x96, 0x30, 0xcb, 0x3d, 0xcc, 0x68, 0x6e, 0x3c, 0x7f, 0x01, 0x95, 0xbd,
  0x7d, 0x63, 0xad, 0x3a, 0xb0, 0xfd, 0xe8, 0xbd, 0xb5, 0x39, 0xb8, 0xbd,
  0x0d, 0xb7, 0x97, 0xbd, 0x5c, 0xbc, 0xd8, 0x3c, 0xba, 0x75, 0x07, 0x3d,
  0x5c, 0xbc, 0x58, 0x3c, 0xb6, 0x08, 0x2c, 0x3e, 0x99, 0xce, 0xb2, 0xbd,
  0x40, 0x51, 0x53, 0xbd, 0xb0, 0xfd, 0xe8, 0x3d, 0xf3, 0x1a, 0x06, 0xbe,
  0x99, 0xce, 0x32, 0xbd, 0xf1, 0x4b, 0x92, 0xbd, 0xba, 0x75, 0x07, 0xbd,
  0x5f, 0x5a, 0xc0, 0x3d, 0x0b, 0x19, 0xb0, 0x3d, 0x0d, 0xb7, 0x97, 0xbd,
  0x9e, 0x0a, 0x82, 0x3d, 0xed, 0x0f, 0xc3, 0x3c, 0xb0, 0xfd, 0x68, 0x3d,
  0xb0, 0xfd, 0x68, 0xbd, 0xb0, 0xfd, 0xe8, 0xbd, 0x94, 0x92, 0x63, 0xbd,
  0x7d, 0x63, 0x2d, 0xbc, 0x04, 0x3f, 0xf9, 0x3c, 0xb5, 0x39, 0x38, 0xbd,
  0xcc, 0x68, 0xee, 0xbd, 0xb5, 0x39, 0xb8, 0x3c, 0x9e, 0x0a, 0x02, 0xbe,
  0x45, 0x8d, 0xa2, 0xbc, 0xb2, 0x9b, 0xd0, 0x3d, 0x24, 0x1e, 0xa0, 0x3d,
  0x42, 0x2a, 0x7b, 0xbc, 0x3c, 0x18, 0x36, 0xbd, 0x10, 0x36, 0x4f, 0x3d,
  0xf4, 0xb2, 0x96, 0xbd, 0xf4, 0xb2, 0x96, 0x3b, 0xe1, 0xca, 0xc5, 0x3d,
  0x53, 0x89, 0x29, 0x3d, 0x8c, 0x68, 0x1b, 0x3e, 0x58, 0x9b, 0xee, 0xbc,
  0x82, 0xf4, 0xb2, 0xbd, 0x58, 0x9b, 0x6e, 0x3d, 0x42, 0x2a, 0x7b, 0x3c,
  0x9b, 0xee, 0xc8, 0xba, 0x85, 0x7d, 0x55, 0x3d, 0xe1, 0xca, 0xc5, 0x3d,
  0x69, 0xfa, 0x9c, 0x3d, 0x69, 0xfa, 0x9c, 0xbd, 0x50, 0x00, 0x87, 0xbd,
  0x26, 0xa7, 0xc2, 0x3d, 0x58, 0x9b, 0xee, 0xbc, 0xcd, 0xe2, 0x74, 0x3d,
  0x24, 0x1e, 0xa0, 0xbd, 0xf4, 0xb2, 0x96, 0x3d, 0x9b, 0xee, 0xc8, 0xbc,
  0x82, 0xf4, 0xb2, 0x3d, 0x96, 0xdc, 0x83, 0xbd, 0x6e, 0x0c, 0x62, 0x3d,
  0x69, 0xfa, 0x1c, 0xbe, 0xfa, 0xc4, 0x5b, 0x3d, 0x7f, 0x6b, 0x10, 0xbd,
  0xf4, 0xb2, 0x96, 0xbc, 0x6c, 0x83, 0xbf, 0xbd, 0x55, 0x12, 0xcc, 0xbd,
  0x69, 0xfa, 0x1c, 0x3e, 0x00, 0x00, 0x00, 0x00, 0x42, 0x2a, 0xfb, 0x3c,
  0xdb, 0xb8, 0x80, 0xbd, 0x9b, 0xee, 0x48, 0x3b, 0x10, 0x36, 0x4f, 0x3d,
  0x12, 0xbf, 0xf1, 0xbd, 0xc8, 0xd0, 0x2f, 0x3c, 0xb1, 0x5f, 0xbc, 0x3c,
  0x69, 0xfa, 0x1c, 0xbd, 0x3a, 0x8f, 0x93, 0xbd, 0xf4, 0xb2, 0x96, 0xbc,
  0x9b, 0xee, 0x48, 0x3c, 0xcd, 0xe2, 0xf4, 0x3d, 0x6e, 0x0c, 0x62, 0xbc,
  0x58, 0x9b, 0x6e, 0xbd, 0xf4, 0xb2, 0x16, 0xbd, 0xf4, 0xb2, 0x16, 0xbc,
  0x0b, 0x24, 0x8a, 0xbd, 0x69, 0xfa, 0x9c, 0xbd, 0x9b, 0xee, 0xc8, 0xbd,
  0x6c, 0x83, 0xbf, 0x3d, 0x6e, 0x0c, 0xe2, 0xbc, 0xbe, 0x5c, 0x47, 0xbe,
  0x42, 0x2a, 0xfb, 0x3b, 0x42, 0x2a, 0xfb, 0x3b, 0x87, 0x06, 0xf8, 0xbd,
  0xca, 0x59, 0xd2, 0xbd, 0x7f, 0x6b, 0x10, 0xbd, 0x42, 0x2a, 0xfb, 0x3b,
  0x16, 0x93, 0x24, 0xbe, 0x8c, 0xa1, 0x82, 0xbe, 0xe4, 0x49, 0x1a, 0x3e,
  0x56, 0x0f, 0x12, 0x3e, 0x88, 0x58, 0x1c, 0x3d, 0x59, 0xbf, 0xf2, 0xbd,
  0xf7, 0x2c, 0x5e, 0x3d, 0x4d, 0x7d, 0xc5, 0xbd, 0xcb, 0x84, 0x6a, 0xbe,
  0x4d, 0x7d, 0xc5, 0xbb, 0xfa, 0x1d, 0x14, 0x3d, 0xa3, 0xcd, 0x2c, 0x3d,
  0x53, 0x1e, 0x5c, 0xbe, 0x94, 0x9a, 0xc9, 0x3d, 0x47, 0xdc, 0x2e, 0xbe,
  0xc8, 0xd4, 0x09, 0xbe, 0x69, 0xf2, 0xd5, 0xbc, 0x41, 0x3b, 0x98, 0xbd,
  0x84, 0x67, 0x66, 0xbc, 0xdb, 0xb7, 0x4d, 0xbd, 0xdb, 0xb7, 0xcd, 0xbd,
  0x6c, 0xe3, 0x8b, 0x3d, 0x69, 0xf2, 0xd5, 0x3d, 0x2e, 0x17, 0xff, 0x3d,
  0x88, 0x58, 0x9c, 0x3d, 0x4d, 0x7d, 0x45, 0x3c, 0x16, 0x93, 0x24, 0x3c,
  0xcf, 0x75, 0xa0, 0xbd, 0xfa, 0x1d, 0x94, 0x3c, 0x25, 0xc6, 0x87, 0x3d,
  0xdb, 0xb7, 0xcd, 0x3d, 0x4d, 0x7d, 0xc5, 0xbc, 0xc5, 0xe3, 0x53, 0x3e,
  0x41, 0x3b, 0x98, 0x3d, 0x16, 0x93, 0x24, 0x3c, 0xde, 0xa8, 0x83, 0x3d,
  0x5c, 0xb0, 0xa8, 0xbd, 0x84, 0x67, 0x66, 0xbd, 0xde, 0xa8, 0x03, 0xbd,
  0x31, 0x08, 0x35, 0xbd, 0xde, 0xa8, 0x83, 0x3d, 0xfa, 0x1d, 0x94, 0xbc,
  0xde, 0xa8, 0x03, 0xbd, 0xcb, 0x84, 0x6a, 0xbe, 0xde, 0xa8, 0x03, 0xbc,
  0xde, 0xa8, 0x03, 0xbd, 0xde, 0xa8, 0x03, 0x3c, 0xfa, 0x1d, 0x14, 0xbe,
  0x4d, 0x7d, 0x45, 0x3d, 0x6c, 0xe3, 0x0b, 0x3d, 0xde, 0xa8, 0x03, 0xbb,
  0x81, 0xb7, 0x05, 0xbe, 0x4d, 0x7d, 0xc5, 0xbc, 0x4d, 0x7d, 0x45, 0x3d,
  0xa3, 0xcd, 0x2c, 0xbd, 0xf7, 0x2c, 0x5e, 0xbd, 0xa3, 0xcd, 0x2c, 0x3d,
  0x4d, 0x7d, 0x45, 0x3c, 0x78, 0x25, 0xb9, 0x3d, 0xbf, 0x42, 0x3d, 0xbd,
  0x6c, 0xe3, 0x0b, 0xbd, 0x4d, 0x7d, 0x45, 0x3d, 0xde, 0xa8, 0x83, 0x3c,
  0xde, 0xa8, 0x83, 0xbd, 0x9b, 0x13, 0x42, 0x3e, 0x1d, 0xb4, 0x92, 0xbb,
  0x7e, 0xd4, 0x4f, 0x3d, 0x13, 0xc6, 0xcc, 0xbd, 0x1d, 0xb4, 0x92, 0xbb,
  0x0c, 0x19, 0xa8, 0x3d, 0xa8, 0xb7, 0x49, 0x3d, 0xfa, 0x7d, 0xbd, 0xbd,
  0xdb, 0x88, 0x89, 0x3d, 0x70, 0x7a, 0x86, 0xbc, 0x70, 0x7a, 0x86, 0xbc,
  0x70, 0x7a, 0x06, 0x3d, 0x2b, 0x0e, 0xdc, 0x3c, 0x66, 0x8c, 0xc0, 0x3d,
  0x5c, 0x9e, 0x7a, 0x3d, 0x2b, 0x0e, 0xdc, 0x3d, 0x99, 0x5d, 0x00, 0x3d,
  0x7e, 0xd4, 0xcf, 0x3c, 0xca, 0xed, 0x1e, 0xbe, 0x2b, 0x0e, 0x5c, 0xbc,
  0x70, 0x7a, 0x86, 0xbc, 0x24, 0x61, 0xb7, 0xbc, 0x77, 0x27, 0xab, 0xbc,
  0xd1, 0x9a, 0xc3, 0xba, 0xf3, 0xd0, 0x98, 0xbd, 0x3c, 0xa9, 0xc6, 0x3d,
  0xd8, 0x47, 0xe8, 0xbc, 0x85, 0x81, 0x74, 0xbd, 0x2b, 0x0e, 0xdc, 0x3d,
  0x88, 0xc2, 0x95, 0x3d, 0x5f, 0xdf, 0x9b, 0xbd, 0xa0, 0x0a, 0xa5, 0xbd,
  0xcf, 0xe4, 0x01, 0xbe, 0x77, 0x27, 0x2b, 0x3d, 0x7e, 0xd4, 0xcf, 0x3c,
  0x70, 0x7a, 0x86, 0xbc, 0x13, 0xc6, 0xcc, 0x3d, 0x35, 0xfc, 0xa1, 0x3d,
  0x4d, 0x44, 0xb1, 0x3d, 0x7e, 0xd4, 0xcf, 0xbc, 0x0c, 0x19, 0xa8, 0xbd,
  0x5c, 0x9e, 0xfa, 0xbd, 0xd1, 0x9a, 0x43, 0xbb, 0x77, 0x27, 0x2b, 0x3c,
  0x7e, 0xd4, 0x4f, 0x3d, 0x2b, 0x0e, 0xdc, 0x3c, 0xa0, 0x0a, 0xa5, 0x3d,
  0xd1, 0x9a, 0x43, 0x3d, 0xf3, 0xd0, 0x18, 0xbd, 0xd1, 0x9a, 0xc3, 0x3c,
  0x1d, 0xb4, 0x92, 0x3b, 0x1d, 0xb4, 0x92, 0xbb, 0x0c, 0x19, 0xa8, 0xbd,
  0xd1, 0x9a, 0x43, 0xbb, 0xd1, 0x9a, 0x43, 0x3d, 0x02, 0x2b, 0xe2, 0xbd,
  0x2b, 0x0e, 0x5c, 0xbc, 0xcf, 0xe4, 0x01, 0xbe, 0x6d, 0x39, 0xe5, 0xbd,
  0xf3, 0xd0, 0x98, 0xbd, 0x77, 0x27, 0xab, 0x3c, 0x7e, 0xd4, 0xcf, 0xbd,
  0xa8, 0xb7, 0x49, 0xbd, 0x4d, 0x44, 0xb1, 0xbd, 0x11, 0xe7, 0x8f, 0xbd,
  0xd9, 0xc7, 0x94, 0xbd, 0x05, 0x19, 0x1c, 0xbd, 0x9f, 0xc1, 0x51, 0xbd,
  0x9f, 0xc1, 0x51, 0xbd, 0xd9, 0xc7, 0x14, 0xbe, 0xd3, 0xe0, 0x1a, 0xbe,
  0x95, 0xda, 0x25, 0xbd, 0x50, 0x06, 0x6f, 0x3d, 0xd7, 0xe0, 0x4c, 0xbd,
  0xd7, 0xe0, 0xcc, 0xbc, 0x88, 0x25, 0xea, 0xbc, 0x18, 0xe7, 0xf3, 0x3c,
  0x31, 0x6a, 0xa3, 0xbd, 0xee, 0x7c, 0x34, 0xbd, 0xa8, 0xa8, 0x7d, 0xbc,
  0xa8, 0xa8, 0x7d, 0xbc, 0x67, 0xa2, 0x56, 0xbc, 0x95, 0xda, 0x25, 0xbd,
  0x3d, 0x38, 0x17, 0xbd, 0x86, 0x0c, 0x00, 0x3e, 0xa8, 0xa8, 0x7d, 0x3c,
  0x26, 0x9c, 0x2f, 0x3d, 0xe4, 0x95, 0x88, 0x3d, 0x05, 0x19, 0x9c, 0xbb,
  0x50, 0x06, 0xef, 0xbd, 0xf7, 0x63, 0xe0, 0x3c, 0x05, 0x19, 0x9c, 0x3a,
  0x05, 0x19, 0x9c, 0xba, 0x50, 0x06, 0x6f, 0xbd, 0xa8, 0xa8, 0xfd, 0x3c,
  0x69, 0x89, 0x9e, 0x3d, 0x05, 0x19, 0x9c, 0x3a, 0x2f, 0x83, 0x5b, 0xbd,
  0x69, 0x89, 0x9e, 0x3d, 0x88, 0x25, 0xea, 0x3b, 0xa7, 0x8f, 0x13, 0x3e,
  0x88, 0x25, 0xea, 0x3b, 0x18, 0xe7, 0xf3, 0xbc, 0x05, 0x19, 0x1c, 0xbd,
  0xbf, 0x44, 0xe5, 0xbd, 0xf7, 0x63, 0xe0, 0xbc, 0xb6, 0x5d, 0xb9, 0x3d,
  0x7e, 0x3e, 0x3e, 0xbd, 0x05, 0x19, 0x9c, 0x3a, 0xe4, 0x95, 0x08, 0x3d,
  0xee, 0x7c, 0x34, 0xbd, 0x67, 0xa2, 0xd6, 0x3d, 0x88, 0x25, 0x6a, 0xbb,
  0x46, 0x1f, 0xc3, 0x3c, 0xa8, 0xa8, 0x7d, 0xbc, 0x05, 0x19, 0x9c, 0xbd,
  0x5d, 0xbb, 0xaa, 0x3d, 0xe4, 0x95, 0x08, 0xbc, 0xcd, 0xf9, 0x20, 0xbd,
  0xbf, 0x44, 0xe5, 0x3d, 0x3b, 0x51, 0xcf, 0x3d, 0x88, 0x25, 0x6a, 0xbc,
  0x26, 0x9c, 0x2f, 0x3c, 0x80, 0x25, 0x86, 0x3d, 0xb6, 0x5d, 0xb9, 0x3c,
  0x7c, 0x57, 0xf6, 0x3d, 0x05, 0x19, 0x1c, 0xbb, 0x31, 0x6a, 0xa3, 0xbd,
  0x11, 0x46, 0xc3, 0xbd, 0x9c, 0x76, 0x2f, 0x3d, 0xf2, 0x41, 0x09, 0xbe,
  0x07, 0x41, 0xd4, 0xbd, 0x99, 0x1f, 0xb5, 0xbb, 0x2e, 0x55, 0x90, 0x3d,
  0x6f, 0xb4, 0x7e, 0x3d, 0xa6, 0x7b, 0x1e, 0xbe, 0x7f, 0x67, 0x62, 0xbc,
  0x2e, 0x55, 0x90, 0xbd, 0x2b, 0xfe, 0x95, 0xbd, 0x72, 0x0b, 0xf9, 0xbc,
  0x1e, 0xa2, 0xac, 0x3d, 0x00, 0x00, 0x00, 0x00, 0x0e, 0xef, 0xc8, 0xbd,
  0x86, 0x15, 0x57, 0xbd, 0xac, 0x29, 0x93, 0x3d, 0x72, 0x0b, 0x79, 0x3c,
  0x82, 0xbe, 0xdc, 0x3d, 0x99, 0x1f, 0xb5, 0xbd, 0x74, 0x6d, 0x06, 0x3e,
  0x04, 0xea, 0xd9, 0x3d, 0x72, 0x0b, 0xf9, 0x3d, 0xa6, 0x7b, 0x1e, 0x3d,
  0x82, 0xbe, 0xdc, 0xbd, 0x74, 0x6d, 0x06, 0xbe, 0x9f, 0xcd, 0x29, 0xbe,
  0x99, 0x1f, 0xb5, 0x3c, 0x72, 0x0b, 0xf9, 0x3d, 0xf4, 0x36, 0xf6, 0xbd,
  0x2b, 0xfe, 0x15, 0x3e, 0x9c, 0x76, 0x2f, 0xbd, 0x0b, 0x98, 0xce, 0xbd,
  0x99, 0x1f, 0x35, 0xbd, 0x7c, 0x10, 0x68, 0xbd, 0xa9, 0xd2, 0x98, 0xbd,
  0x82, 0xbe, 0x5c, 0x3d, 0x7f, 0x67, 0x62, 0x3c, 0x7f, 0x67, 0x62, 0x3c,
  0x7f, 0x67, 0xe2, 0xbd, 0x6f, 0xb4, 0x7e, 0x3d, 0x8c, 0xc3, 0x4b, 0xbc,
  0x99, 0x1f, 0xb5, 0x3b, 0xb3, 0xd7, 0x87, 0x3d, 0x5a, 0xb5, 0x33, 0x3e,
  0x2b, 0xfe, 0x95, 0xbd, 0x7f, 0x67, 0xe2, 0x3d, 0x72, 0x0b, 0xf9, 0xbc,
  0x00, 0x00, 0x00, 0x00, 0x82, 0xbe, 0xdc, 0xbd, 0xec, 0x93, 0x14, 0xbe,
  0x7c, 0x10, 0xe8, 0xbd, 0xa6, 0x7b, 0x1e, 0x3c, 0xa9, 0xd2, 0x18, 0x3d,
  0x99, 0x1f, 0xb5, 0xba, 0xb3, 0xd7, 0x87, 0xbb, 0x7c, 0x10, 0x68, 0x3d,
  0xb3, 0xd7, 0x07, 0xbd, 0x89, 0x6c, 0x51, 0xbd, 0xac, 0x29, 0x93, 0xbc,
  0x00, 0x00, 0x00, 0x00, 0x8c, 0xc3, 0x4b, 0x3d, 0xa6, 0x7b, 0x9e, 0xbd,
  0x9f, 0xcd, 0xa9, 0xbd, 0xa7, 0xb6, 0x16, 0x3e, 0xc1, 0x12, 0x34, 0xbd,
  0x0c, 0xe7, 0x3b, 0x3c, 0xa3, 0x8f, 0xcb, 0xbc, 0x14, 0x4e, 0x57, 0xbe,
  0x66, 0x89, 0x7a, 0xbb, 0xc1, 0x12, 0x34, 0xbe, 0xaa, 0xf6, 0xe6, 0xbd,
  0xee, 0x63, 0x53, 0xbd, 0x49, 0xed, 0x8c, 0x3c, 0x76, 0x3e, 0xac, 0xbd,
  0x0c, 0xe7, 0x3b, 0x3d, 0x95, 0xc1, 0x94, 0xbd, 0x66, 0x89, 0xfa, 0xbc,
  0x66, 0x89, 0x7a, 0x3b, 0x32, 0xd1, 0x3f, 0xbe, 0xe0, 0x95, 0x1c, 0xbc,
  0x37, 0xf8, 0x0a, 0x3e, 0x39, 0x38, 0xdb, 0x3c, 0xc1, 0x12, 0x34, 0x3d,
  0xaa, 0xf6, 0xe6, 0xbd, 0x39, 0x38, 0xdb, 0x3c, 0xba, 0xab, 0x98, 0xbd,
  0x49, 0xed, 0x8c, 0xbc, 0x2b, 0x6a, 0x24, 0x3d, 0xe0, 0x95, 0x9c, 0x3d,
  0xee, 0x63, 0xd3, 0x3d, 0x32, 0xd1, 0xbf, 0xbd, 0xee, 0x63, 0x53, 0xbd,
  0xee, 0x63, 0xd3, 0x3d, 0x24, 0x03, 0x09, 0xbe, 0xd4, 0x07, 0x36, 0xbe,
  0x49, 0xed, 0x0c, 0x3d, 0x32, 0xd1, 0xbf, 0x3d, 0xee, 0x63, 0x53, 0xbd,
  0xd0, 0xe0, 0xea, 0xbd, 0xa3, 0x8f, 0xcb, 0xbc, 0xd0, 0xe0, 0xea, 0xbc,
  0xe0, 0x95, 0x9c, 0xbc, 0xd0, 0xe0, 0x6a, 0x3d, 0xc1, 0x12, 0x34, 0x3d,
  0x66, 0x89, 0xfa, 0x3c, 0xaf, 0x1d, 0x32, 0xbe, 0x0c, 0xe7, 0xbb, 0xbc,
  0xfe, 0x18, 0x05, 0x3d, 0xe7, 0xfc, 0xb7, 0x3d, 0x66, 0x89, 0x7a, 0x3d,
  0x0c, 0xe7, 0x3b, 0x3d, 0x24, 0x03, 0x09, 0xbe, 0xfe, 0x18, 0x05, 0x3d,
  0x51, 0x54, 0xa8, 0xbd, 0x39, 0x38, 0xdb, 0x3c, 0x53, 0x94, 0x78, 0x3e,
  0xe0, 0x95, 0x1c, 0x3c, 0xd0, 0xe0, 0xea, 0x3d, 0xee, 0x63, 0x53, 0xbd,
  0xfe, 0x18, 0x05, 0x3d, 0x84, 0x0c, 0xe3, 0xbd, 0x49, 0xed, 0x0c, 0xbe,
  0x1b, 0xb5, 0x72, 0x3d, 0x45, 0xc6, 0x41, 0x3e, 0xe0, 0x95, 0x9c, 0x3d,
  0x95, 0xc1, 0x14, 0xbd, 0xa3, 0x8f, 0xcb, 0x3c, 0x08, 0xf5, 0xff, 0xff,
  0x0c, 0xf5, 0xff, 0xff, 0x0f, 0x00, 0x00, 0x00, 0x4d, 0x4c, 0x49, 0x52,
  0x20, 0x43, 0x6f, 0x6e, 0x76, 0x65, 0x72, 0x74, 0x65, 0x64, 0x2e, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00, 0x00, 0x00, 0x0e, 0x00,
  0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x08, 0x00, 0x04, 0x00,
  0x0e, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0xf8, 0x01, 0x00, 0x00, 0xfc, 0x01, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x6d, 0x61, 0x69, 0x6e, 0x00, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x00, 0x00, 0xa4, 0x01, 0x00, 0x00, 0x40, 0x01, 0x00, 0x00,
  0x0c, 0x01, 0x00, 0x00, 0xac, 0x00, 0x00, 0x00, 0x84, 0x00, 0x00, 0x00,
  0x4c, 0x00, 0x00, 0x00, 0x28, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x16, 0xff, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x07, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x0e, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00, 0x36, 0xff, 0xff, 0xff,
  0x0c, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00, 0x06, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x2a, 0xff, 0xff, 0xff, 0x14, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x08, 0x10, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x05, 0x00, 0x00, 0x00, 0xdc, 0xf5, 0xff, 0xff, 0x01, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 0x0a, 0x00, 0x00, 0x00,
  0x0b, 0x00, 0x00, 0x00, 0x05, 0x00, 0x00, 0x00, 0x8a, 0xff, 0xff, 0xff,
  0x0c, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x0a, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x09, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 0x82, 0xff, 0xff, 0xff,
  0x24, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x05, 0x34, 0x00, 0x00, 0x00,
  0x38, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 0x00, 0x00, 0x0e, 0x00,
  0x18, 0x00, 0x17, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x08, 0x00, 0x04, 0x00,
  0x0e, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01,
  0x01, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x00, 0x00, 0x00, 0x00, 0x0a, 0x00, 0x10, 0x00, 0x0c, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x0a, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x10, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x0e, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00,
  0x0b, 0x00, 0x04, 0x00, 0x0e, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x01, 0x2c, 0x00, 0x00, 0x00, 0x30, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x14, 0x00, 0x13, 0x00, 0x0c, 0x00,
  0x08, 0x00, 0x07, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01,
  0x02, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01,
  0x01, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x06, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x0e, 0x00, 0x14, 0x00, 0x00, 0x00, 0x10, 0x00, 0x0c, 0x00,
  0x0b, 0x00, 0x04, 0x00, 0x0e, 0x00, 0x00, 0x00, 0x18, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x4a, 0x18, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x08, 0x00, 0x00, 0x00, 0x07, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x01, 0x01, 0x00, 0x00, 0x00, 0x06, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x0e, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x7c, 0x08, 0x00, 0x00,
  0xc4, 0x07, 0x00, 0x00, 0x5c, 0x07, 0x00, 0x00, 0xfc, 0x06, 0x00, 0x00,
  0xa0, 0x06, 0x00, 0x00, 0x14, 0x06, 0x00, 0x00, 0x94, 0x05, 0x00, 0x00,
  0xbc, 0x04, 0x00, 0x00, 0x64, 0x03, 0x00, 0x00, 0xb8, 0x02, 0x00, 0x00,
  0x3c, 0x02, 0x00, 0x00, 0x94, 0x01, 0x00, 0x00, 0xe4, 0x00, 0x00, 0x00,
  0x60, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xd6, 0xf7, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x01, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x34, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff, 0x01, 0x00, 0x00, 0x00,
  0xb8, 0xf7, 0xff, 0xff, 0x19, 0x00, 0x00, 0x00, 0x53, 0x74, 0x61, 0x74,
  0x65, 0x66, 0x75, 0x6c, 0x50, 0x61, 0x72, 0x74, 0x69, 0x74, 0x69, 0x6f,
  0x6e, 0x65, 0x64, 0x43, 0x61, 0x6c, 0x6c, 0x3a, 0x30, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x26, 0xfd, 0xff, 0xff, 0x00, 0x00, 0x00, 0x01, 0x18, 0x00, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00, 0x0e, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x09, 0x58, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0xff, 0xff, 0xff, 0xff, 0x01, 0x00, 0x00, 0x00, 0x7c, 0xfa, 0xff, 0xff,
  0x08, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x80, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0x00, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x80, 0x3b, 0x1d, 0x00, 0x00, 0x00,
  0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e,
  0x74, 0x5f, 0x64, 0x65, 0x6e, 0x73, 0x65, 0x5f, 0x33, 0x2f, 0x53, 0x69,
  0x67, 0x6d, 0x6f, 0x69, 0x64, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0xa6, 0xfd, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x01, 0x18, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00,
  0x40, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x09,
  0x74, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff,
  0x01, 0x00, 0x00, 0x00, 0xfc, 0xfa, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0xdc, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x8b, 0x82, 0x04, 0x3e, 0x3a, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65,
  0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x64, 0x65,
  0x6e, 0x73, 0x65, 0x5f, 0x33, 0x2f, 0x4d, 0x61, 0x74, 0x4d, 0x75, 0x6c,
  0x3b, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61,
  0x6e, 0x74, 0x5f, 0x64, 0x65, 0x6e, 0x73, 0x65, 0x5f, 0x33, 0x2f, 0x42,
  0x69, 0x61, 0x73, 0x41, 0x64, 0x64, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x0e, 0x00,
  0x18, 0x00, 0x14, 0x00, 0x13, 0x00, 0x0c, 0x00, 0x08, 0x00, 0x04, 0x00,
  0x0e, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00, 0x30, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x09, 0x84, 0x00, 0x00, 0x00,
  0x94, 0xfb, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x02, 0xcd, 0xec, 0x3b, 0x59, 0x00, 0x00, 0x00,
  0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e,
  0x74, 0x5f, 0x64, 0x65, 0x6e, 0x73, 0x65, 0x5f, 0x33, 0x2f, 0x4d, 0x61,
  0x74, 0x4d, 0x75, 0x6c, 0x3b, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31,
  0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x64, 0x65, 0x6e, 0x73, 0x65,
  0x5f, 0x33, 0x2f, 0x4c, 0x61, 0x73, 0x74, 0x56, 0x61, 0x6c, 0x75, 0x65,
  0x51, 0x75, 0x61, 0x6e, 0x74, 0x2f, 0x46, 0x61, 0x6b, 0x65, 0x51, 0x75,
  0x61, 0x6e, 0x74, 0x57, 0x69, 0x74, 0x68, 0x4d, 0x69, 0x6e, 0x4d, 0x61,
  0x78, 0x56, 0x61, 0x72, 0x73, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x20, 0x01, 0x00, 0x00, 0xf6, 0xfe, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x01, 0x18, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x0b, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x09,
  0x50, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff,
  0x20, 0x01, 0x00, 0x00, 0x4c, 0xfc, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00,
  0x10, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x84, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0x01, 0x00, 0x00, 0x00, 0x60, 0xfd, 0x76, 0x3d,
  0x19, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f,
  0x66, 0x6c, 0x61, 0x74, 0x74, 0x65, 0x6e, 0x5f, 0x32, 0x2f, 0x52, 0x65,
  0x73, 0x68, 0x61, 0x70, 0x65, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x20, 0x01, 0x00, 0x00, 0x6e, 0xff, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x01, 0x18, 0x00, 0x00, 0x00, 0x28, 0x00, 0x00, 0x00,
  0x48, 0x00, 0x00, 0x00, 0x0a, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x09,
  0x60, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff,
  0x06, 0x00, 0x00, 0x00, 0x06, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0xcc, 0xfc, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x84, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x60, 0xfd, 0x76, 0x3d,
  0x1f, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f,
  0x6d, 0x61, 0x78, 0x5f, 0x70, 0x6f, 0x6f, 0x6c, 0x69, 0x6e, 0x67, 0x32,
  0x64, 0x5f, 0x32, 0x2f, 0x4d, 0x61, 0x78, 0x50, 0x6f, 0x6f, 0x6c, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x06, 0x00, 0x00, 0x00,
  0x06, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0x00, 0x00, 0x16, 0x00,
  0x20, 0x00, 0x1c, 0x00, 0x1b, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x08, 0x00, 0x07, 0x00, 0x16, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x01, 0x18, 0x00, 0x00, 0x00, 0x28, 0x00, 0x00, 0x00,
  0x48, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x09,
  0x24, 0x01, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff,
  0x0d, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x74, 0xfd, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x84, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x60, 0xfd, 0x76, 0x3d,
  0xe2, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f,
  0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x63, 0x6f, 0x6e, 0x76, 0x32, 0x64,
  0x5f, 0x32, 0x2f, 0x4d, 0x6f, 0x76, 0x69, 0x6e, 0x67, 0x41, 0x76, 0x67,
  0x51, 0x75, 0x61, 0x6e, 0x74, 0x69, 0x7a, 0x65, 0x2f, 0x46, 0x61, 0x6b,
  0x65, 0x51, 0x75, 0x61, 0x6e, 0x74, 0x57, 0x69, 0x74, 0x68, 0x4d, 0x69,
  0x6e, 0x4d, 0x61, 0x78, 0x56, 0x61, 0x72, 0x73, 0x3b, 0x6d, 0x6f, 0x64,
  0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x63,
  0x6f, 0x6e, 0x76, 0x32, 0x64, 0x5f, 0x32, 0x2f, 0x4d, 0x6f, 0x76, 0x69,
  0x6e, 0x67, 0x41, 0x76, 0x67, 0x51, 0x75, 0x61, 0x6e, 0x74, 0x69, 0x7a,
  0x65, 0x2f, 0x46, 0x61, 0x6b, 0x65, 0x51, 0x75, 0x61, 0x6e, 0x74, 0x57,
  0x69, 0x74, 0x68, 0x4d, 0x69, 0x6e, 0x4d, 0x61, 0x78, 0x56, 0x61, 0x72,
  0x73, 0x2f, 0x52, 0x65, 0x61, 0x64, 0x56, 0x61, 0x72, 0x69, 0x61, 0x62,
  0x6c, 0x65, 0x4f, 0x70, 0x3b, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31,
  0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x63, 0x6f, 0x6e, 0x76, 0x32,
  0x64, 0x5f, 0x32, 0x2f, 0x4d, 0x6f, 0x76, 0x69, 0x6e, 0x67, 0x41, 0x76,
  0x67, 0x51, 0x75, 0x61, 0x6e, 0x74, 0x69, 0x7a, 0x65, 0x2f, 0x46, 0x61,
  0x6b, 0x65, 0x51, 0x75, 0x61, 0x6e, 0x74, 0x57, 0x69, 0x74, 0x68, 0x4d,
  0x69, 0x6e, 0x4d, 0x61, 0x78, 0x56, 0x61, 0x72, 0x73, 0x2f, 0x52, 0x65,
  0x61, 0x64, 0x56, 0x61, 0x72, 0x69, 0x61, 0x62, 0x6c, 0x65, 0x4f, 0x70,
  0x5f, 0x31, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x0d, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x72, 0xfc, 0xff, 0xff, 0x00, 0x00, 0x00, 0x01, 0x14, 0x00, 0x00, 0x00,
  0x24, 0x00, 0x00, 0x00, 0x24, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0xa8, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff,
  0x0d, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x5c, 0xfc, 0xff, 0xff, 0x86, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65,
  0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x63, 0x6f,
  0x6e, 0x76, 0x32, 0x64, 0x5f, 0x32, 0x2f, 0x52, 0x65, 0x6c, 0x75, 0x3b,
  0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e,
  0x74, 0x5f, 0x63, 0x6f, 0x6e, 0x76, 0x32, 0x64, 0x5f, 0x32, 0x2f, 0x42,
  0x69, 0x61, 0x73, 0x41, 0x64, 0x64, 0x3b, 0x6d, 0x6f, 0x64, 0x65, 0x6c,
  0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x63, 0x6f, 0x6e,
  0x76, 0x32, 0x64, 0x5f, 0x32, 0x2f, 0x43, 0x6f, 0x6e, 0x76, 0x32, 0x44,
  0x3b, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61,
  0x6e, 0x74, 0x5f, 0x63, 0x6f, 0x6e, 0x76, 0x32, 0x64, 0x5f, 0x32, 0x2f,
  0x42, 0x69, 0x61, 0x73, 0x41, 0x64, 0x64, 0x2f, 0x52, 0x65, 0x61, 0x64,
  0x56, 0x61, 0x72, 0x69, 0x61, 0x62, 0x6c, 0x65, 0x4f, 0x70, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00,
  0x0d, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0x46, 0xfd, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x01, 0x14, 0x00, 0x00, 0x00, 0x24, 0x00, 0x00, 0x00,
  0x24, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00, 0x50, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff, 0x20, 0x00, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x30, 0xfd, 0xff, 0xff,
  0x2f, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f,
  0x72, 0x65, 0x73, 0x69, 0x7a, 0x69, 0x6e, 0x67, 0x5f, 0x32, 0x2f, 0x72,
  0x65, 0x73, 0x69, 0x7a, 0x65, 0x2f, 0x52, 0x65, 0x73, 0x69, 0x7a, 0x65,
  0x4e, 0x65, 0x61, 0x72, 0x65, 0x73, 0x74, 0x4e, 0x65, 0x69, 0x67, 0x68,
  0x62, 0x6f, 0x72, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0xda, 0xfe, 0xff, 0xff, 0x00, 0x00, 0x00, 0x01, 0x20, 0x00, 0x00, 0x00,
  0x40, 0x00, 0x00, 0x00, 0x06, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x02,
  0x68, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x60, 0x77, 0xe4, 0x39, 0x2c, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65,
  0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x64, 0x65,
  0x6e, 0x73, 0x65, 0x5f, 0x33, 0x2f, 0x42, 0x69, 0x61, 0x73, 0x41, 0x64,
  0x64, 0x2f, 0x52, 0x65, 0x61, 0x64, 0x56, 0x61, 0x72, 0x69, 0x61, 0x62,
  0x6c, 0x65, 0x4f, 0x70, 0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0xfe, 0xfe, 0xff, 0xff, 0x00, 0x00, 0x00, 0x01,
  0x10, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00, 0x05, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x1c, 0xfe, 0xff, 0xff, 0x2d, 0x00, 0x00, 0x00,
  0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e,
  0x74, 0x5f, 0x63, 0x6f, 0x6e, 0x76, 0x32, 0x64, 0x5f, 0x32, 0x2f, 0x42,
  0x69, 0x61, 0x73, 0x41, 0x64, 0x64, 0x2f, 0x52, 0x65, 0x61, 0x64, 0x56,
  0x61, 0x72, 0x69, 0x61, 0x62, 0x6c, 0x65, 0x4f, 0x70, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0xba, 0xff, 0xff, 0xff,
  0x00, 0x00, 0x00, 0x01, 0x14, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x02, 0x24, 0x00, 0x00, 0x00,
  0x78, 0xfe, 0xff, 0xff, 0x17, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65,
  0x6c, 0x5f, 0x31, 0x2f, 0x66, 0x6c, 0x61, 0x74, 0x74, 0x65, 0x6e, 0x5f,
  0x32, 0x2f, 0x43, 0x6f, 0x6e, 0x73, 0x74, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x16, 0x00, 0x1c, 0x00, 0x18, 0x00,
  0x17, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x08, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x07, 0x00, 0x16, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01,
  0x14, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x02, 0x2c, 0x00, 0x00, 0x00, 0xd4, 0xfe, 0xff, 0xff,
  0x1e, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f,
  0x72, 0x65, 0x73, 0x69, 0x7a, 0x69, 0x6e, 0x67, 0x5f, 0x32, 0x2f, 0x72,
  0x65, 0x73, 0x69, 0x7a, 0x65, 0x2f, 0x73, 0x69, 0x7a, 0x65, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x16, 0x00,
  0x18, 0x00, 0x14, 0x00, 0x00, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x08, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x07, 0x00, 0x16, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x01, 0x10, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0x74, 0x00, 0x00, 0x00, 0x34, 0xff, 0xff, 0xff,
  0x65, 0x00, 0x00, 0x00, 0x6d, 0x6f, 0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f,
  0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f, 0x63, 0x6f, 0x6e, 0x76, 0x32, 0x64,
  0x5f, 0x32, 0x2f, 0x43, 0x6f, 0x6e, 0x76, 0x32, 0x44, 0x3b, 0x6d, 0x6f,
  0x64, 0x65, 0x6c, 0x5f, 0x31, 0x2f, 0x71, 0x75, 0x61, 0x6e, 0x74, 0x5f,
  0x63, 0x6f, 0x6e, 0x76, 0x32, 0x64, 0x5f, 0x32, 0x2f, 0x4c, 0x61, 0x73,
  0x74, 0x56, 0x61, 0x6c, 0x75, 0x65, 0x51, 0x75, 0x61, 0x6e, 0x74, 0x2f,
  0x46, 0x61, 0x6b, 0x65, 0x51, 0x75, 0x61, 0x6e, 0x74, 0x57, 0x69, 0x74,
  0x68, 0x4d, 0x69, 0x6e, 0x4d, 0x61, 0x78, 0x56, 0x61, 0x72, 0x73, 0x50,
  0x65, 0x72, 0x43, 0x68, 0x61, 0x6e, 0x6e, 0x65, 0x6c, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x16, 0x00,
  0x1c, 0x00, 0x18, 0x00, 0x00, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00,
  0x00, 0x00, 0x00, 0x00, 0x08, 0x00, 0x07, 0x00, 0x16, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x01, 0x14, 0x00, 0x00, 0x00, 0x28, 0x00, 0x00, 0x00,
  0x28, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xff, 0xff, 0xff, 0xff, 0xf9, 0x00, 0x00, 0x00,
  0x01, 0x01, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x04, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x19, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x5f,
  0x69, 0x6e, 0x70, 0x75, 0x74, 0x5f, 0x33, 0x3a, 0x30, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0xf9, 0x00, 0x00, 0x00,
  0x01, 0x01, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00,
  0x9c, 0x00, 0x00, 0x00, 0x80, 0x00, 0x00, 0x00, 0x64, 0x00, 0x00, 0x00,
  0x50, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00, 0x2c, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x90, 0xff, 0xff, 0xff,
  0x06, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x06,
  0xa0, 0xff, 0xff, 0xff, 0x0e, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x0e, 0xb0, 0xff, 0xff, 0xff, 0x09, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x09, 0xd8, 0xff, 0xff, 0xff,
  0x16, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x16, 0xcc, 0xff, 0xff, 0xff,
  0x11, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x11,
  0xf4, 0xff, 0xff, 0xff, 0x72, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x72,
  0x0c, 0x00, 0x0c, 0x00, 0x0b, 0x00, 0x00, 0x00, 0x00, 0x00, 0x04, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x03,
  0x0c, 0x00, 0x10, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x61, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x00, 0x00, 0x00, 0x61
};
Slide Type
1
​

```